# Calculation of maps of weights

- ESM models: Results of 8 $\textbf{CMIP6}$ models.

- Products based on observations to use as reference: $\textbf{X-BASE}$ (FLUXCOM-X) for NEP and $\textbf{CarboScope}$ for NBP.

This notebook assumes that users have local access to CMIP6 model results. Due to their large size, these datasets are not included but can be downloaded from the portals:

- USA, PCMDI/LLNL (California) -  https://esgf-node.llnl.gov/search/cmip6/

- France, IPSL -  https://esgf-node.ipsl.upmc.fr/search/cmip6-ipsl/

- Germany, DKRZ - https://esgf-data.dkrz.de/search/cmip6-dkrz/

- UK, CEDA - https://esgf-index1.ceda.ac.uk/search/cmip6-ceda/


#### Libraries

In [1]:
from data_compilation import *

## Read data

- CMIP6: 01/1850-12/2014
- X-BASE (FLUXCOM-X):01/2001-12/2021
- CarboScope: 01/1957-12/2023

In [9]:
# choose an observations-based product.
ob='CarboScope' # 'FLUXCOM-X','CarboScope'. Note that FLUXCOM-X only has nee and CarboScope nbp.
mb='CMIP6' 

In [10]:
if ob=='FLUXCOM-X':
    mbv='nep'
    d1="2001-01";d2="2014-12" # CMIP6-FLUXCOM.
    varESM=get_nepCMIP6(d1,d2)[0]
    varObs=get_neeFLUXCOM(d1,d2) # it takes a long time.
    varObs=varObs*(-1) # FlUXCOM-X considers flux into the atmosphere positive.
if ob=='CarboScope':
    mbv='nbp'
    d1="1957-01"; d2="2014-12" # CMIP6-Carboscope.
    varESM=get_nbpCMIP6(d1,d2)  
    varObs=get_nbpCarboscope(d1,d2)*(-1) # CarboScope considers flux into the atmosphere positive.

/home/estefania/anaconda3/envs/xesmf_env/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'nbp' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/estefania/anaconda3/envs/xesmf_env/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'nbp' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/estefania/anaconda3/envs/xesmf_env/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'nbp' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/estefania/anaconda3/envs/xesmf_env/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'nbp' has multiple fill values {np.float32(1e+20), np.fl

## Weights
Calculations based on procedure in described in the attached manuscript.

In [11]:
e2=(varESM-varObs)**2 # (Eq.16) [(kgC m-2 yr-1)^2] # residual for each grid cell and each time point between model i and data product j.
n=len(e2.time)
s2=e2.sum("time",skipna=False)/n # (Eq.17) [(kgC m-2 yr-1)^2] # variance over time
#s2=s2*maskl
A=n*np.log(s2) # (Eq.18) [(kgC m-2 yr-1)^2]. 
Amin=A.to_array("variable").min("variable") # (Eq. 19) grid-cells from this set of maps in which the values of A are the lowest.
D=A-Amin # (Eq.20) differences with respect to Amin minimum.
wm=np.exp(-D/n)/(np.exp(-D/n).to_array("variable").sum("variable")) #(Eq. 21) # maps of weights [-].

In [12]:
Amin.to_netcdf('/home/data/ResultsCS/Weights/'+mbv+mb+'_ob_'+ob+'_Amin.nc')

#### Save results

In [24]:
Amin.to_netcdf('/home/data/ResultsCS/Weights/'+mbv+mb+'_ob_'+ob+'_Amin.nc')
wm.to_netcdf('/home/data/ResultsCS/Weights/'+mbv+mb+'_ob_'+ob+'.nc')
s2.to_netcdf('/home/data/ResultsCS/Weights/'+mbv+mb+'_ob_'+ob+'_variance.nc') # to calculate the variance.